##### The following code analyzes the viable_patches_json file. The points of analysis are described below. The primary tool for this analysis is pydriller.

1. Total size of the cloned repos
2. Total number of vulnerability inducing commits (vuln commits) found & not found
3. Average number of vuln commits to patch commit (or fix)
4. Average number of months between a vuln commit
5. Percentage of vulns where the vuln commit and fix were made by the same person


##### Sources
- @inbook{PyDriller,
    title = "PyDriller: Python Framework for Mining Software Repositories",
    abstract = "Software repositories contain historical and valuable information about the overall development of software systems. Mining software repositories (MSR) is nowadays considered one of the most interesting growing fields within software engineering. MSR focuses on extracting and analyzing data available in software repositories to uncover interesting, useful, and actionable information about the system. Even though MSR plays an important role in software engineering research, few tools have been created and made public to support developers in extracting information from Git repository. In this paper, we present PyDriller, a Python Framework that eases the process of mining Git. We compare our tool against the state-of-the-art Python Framework GitPython, demonstrating that PyDriller can achieve the same results with, on average, 50% less LOC and significantly lower complexity.URL: https://github.com/ishepard/pydrillerMaterials: https://doi.org/10.5281/zenodo.1327363Pre-print: https://doi.org/10.5281/zenodo.1327411",
    author = "Spadini, Davide and Aniche, Maurício and Bacchelli, Alberto",
    year = "2018",
    doi = "10.1145/3236024.3264598",
    booktitle = "The 26th ACM Joint European Software Engineering Conference and Symposium on the Foundations of Software Engineering (ESEC/FSE)",
    }

##### Author @Trust-Worthy



##### Reading in the results from the patch_vuln_match.json file and processing objects according to JSONL standard

In [8]:
import pandas as pd
import jsonlines    

json_path:str = "../production_ready/patch_vuln_match.jsonl"

data: list[object] = []

with jsonlines.open(json_path) as reader:

    data = [entry for entry in reader]

# Convert the list of dictionaries into a pandas DataFrame
patch_vuln_df = pd.DataFrame(data)


# Define a function to extract the file paths and commits
def extract_vuln_files_commits(vuln_commits):
    if vuln_commits:
        files = list(vuln_commits.keys())
        commits = [commit for commits in vuln_commits.values() for commit in commits]
        return pd.Series([files, commits])
    else:
        return pd.Series([[], []])  # Empty lists if no vuln_commits

# Apply the function to create new columns
patch_vuln_df[['vuln_files', 'vuln_commits']] = patch_vuln_df['vuln_commits'].apply(extract_vuln_files_commits)



print(patch_vuln_df.head())


           cve_id                     repo  \
0   CVE-1999-0199             bminor/glibc   
1   CVE-1999-0731         KDE/kde1-kdebase   
2   CVE-2002-2443                krb5/krb5   
3  CVE-2005-10002  wp-plugins/secure-files   
4  CVE-2005-10003      mikexstudios/xcomic   

                               patch_commit  \
0  2864e767053317538feafa815046fff89e5a16be   
1  04906bd5de2f220bf100b605dad37b4a1d9a91a6   
2  cf1a0c411b2668c57c41e9c4efd15ba17b6b322c   
3  cab025e5fc2bcdad8032d833ebc38e6bd2a13c92   
4  6ed8e3cc336e29f09c7e791863d0559939da98bf   

                                        vuln_commits  \
0  [dc5efe83c0252ad45337ab98eff6c26fdb29b0a9, 27a...   
1                                                 []   
2  [e88f857c3680ea395c0bed6a82862d8ea1177221, 438...   
3         [b1afc063fd49cfb875e1c6f591543ebff6649469]   
4                                                 []   

                                          vuln_files  
0  [elf/dl-load.c, manual/search.texi, misc/sysl